In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
genre_names = ['blues', 'classical', 'country', 'disco', 'hipop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

# Dataset import
data = pd.read_csv('extracted_dataset.csv')

# Dataset normalization
data_mean = data.mean()
data_std = data.std()

data_normalized = (data - data_mean) / data_std

dataset = data_normalized.to_numpy()[:, 1:4]
labels = data.to_numpy()[:, 4].astype(int)

# **Separation of the training set and the validation set**

In [4]:
#Indexes extraction
indexes_train = np.random.choice(1000, 800, replace=False)
indexes_valid= np.setdiff1d(np.array([i for i in range(1000)]), indexes_train)

dataset_train = dataset[indexes_train, :]
dataset_valid = dataset[indexes_valid, :]

labels_train = labels[indexes_train]
labels_valid = labels[indexes_valid]

# **Primal linear boundary**

In [ ]:
reg = 0.001
learning_rate = 0.02

w = np.random.randn(3)
b = 0

In [ ]:
def classify(x):
  return (np.dot(w, x) - b)

def hingeloss(x, y):
  prod = y * classify(x)

  if prod >= 1: return 0
  else: return 1-prod

def hinge_derivative_w(x, y):
  prod = y * classify(x)
  prod_elem = y * x
  if prod >= 1: return np.zeros(x.shape[0])
  else: return -prod_elem

def hinge_derivative_b(x, y):
  prod = y * classify(x)
  if prod >= 1: return 0
  else: return y

# **Training**

In [ ]:
history_loss = list()
loss = 0

for epoch in range(1000):
  gradient_w = np.zeros(w.size)
  gradient_b = 0

  for sample_index in range(100):
    gradient_w += hinge_derivative_w(class1_data[sample_index,:], 1)
    gradient_w += hinge_derivative_w(class2_data[sample_index,:], -1)
    gradient_b += hinge_derivative_b(class1_data[sample_index,:], 1)
    gradient_b += hinge_derivative_b(class2_data[sample_index,:], -1)

  gradient_w /= 200.0
  gradient_b /= 200.0

  w = w - learning_rate * gradient_w
  b = b - learning_rate * gradient_b

  loss=0
  for sample_index in range(100):
    loss += hingeloss(class1_data[sample_index,:], 1)
    loss += hingeloss(class2_data[sample_index,:], -1)

  history_loss.append(loss/200)

print(w)
print(b)

fig, axs = plt.subplots(figsize = (16,8))

axs.semilogy(history_loss)

# **Confusion matrix**

In [ ]:
#da fare

# Primal with quadratic feature map

In [ ]:
def phi(x):
  f1 = x[0]
  f2 = x[1]
  f3 = x[2]
  return np.array([f1, f2, f3, f1**2, f2**2, f3**2, f1*f2, f2*f3, f1*f3])

In [ ]:
feature_size = 9
w = np.zeros(feature_size)
b = 0

history_loss = list()
loss = 0

for epoch in range(10000):
  gradient_w = np.zeros(w.size)
  gradient_b = 0
  for sample_index in range(100):
    phi_1 = phi(class1_data[sample_index,:])
    phi_2 = phi(class2_data[sample_index,:])
    gradient_w += hinge_derivative_w(phi_1, 1)
    gradient_w += hinge_derivative_w(phi_2, -1)
    gradient_b += hinge_derivative_b(phi_1, 1)
    gradient_b += hinge_derivative_b(phi_2, -1)
  gradient_w /= 200.0
  gradient_b /= 200.0
  
  w = w - learning_rate * gradient_w
  b = b - learning_rate * gradient_b

  loss=0
  for sample_index in range(100):
    phi_1 = phi(class1_data[sample_index,:])
    phi_2 = phi(class2_data[sample_index,:])
    loss += hingeloss(phi_1, 1)
    loss += hingeloss(phi_2, -1)

  history_loss.append(loss/200)

print(w)
print(b)

fig, axs = plt.subplots(figsize = (16,8))

axs.semilogy(history_loss)

# **Dual with linear margin**

In [ ]:
import jax
import jax.numpy as jnp
import scipy.optimize as opt

In [ ]:
def hessian(x):
  return np.zeros((200, 200))

def lag(c):
  partial_1 = - jnp.sum(c)
  X_1 = in_data * in_labels[:, None]
  H = X_1 @ X_1.T
  partial_2 = (c.T @ H @ c)/2
  return partial_1 + partial_2

In [ ]:
c_initial = np.zeros(200)
in_data = np.concatenate((class1_data, class2_data), axis=0)
in_labels = np.concatenate((np.ones(100), -1 * np.ones(100)), axis=None)

lag_jit=jax.jit(lag)

linear_constraint = opt.LinearConstraint(in_labels, 0, 0, keep_feasible=True)
res = opt.minimize(lag_jit, c_initial, method='trust-constr', jac='2-point', hess=hessian, constraints=[linear_constraint], options={'verbose': 1, 'maxiter': 10000}, bounds=opt.Bounds(np.zeros(200), np.ones(200)*np.inf))

In [ ]:
c = np.array(res.x)

w = np.zeros(3)

for sample_index in range(200):
  w += c[sample_index] * in_labels[sample_index] * in_data[sample_index, :]

best_b = 0
best_prec = 0

for i in range(200):
  b = np.dot(w, in_data[i, :] - in_labels[i])
  confusion_mat = np.zeros((2,2))
  for test_index in range(100):
    predicted = 0 if classify(class1_data[test_index,:]) >= 0 else 1
    confusion_mat[0,predicted] += 1
    predicted = 0 if classify(class2_data[test_index,:]) >= 0 else 1
    confusion_mat[1,predicted] += 1
    prec = confusion_mat.trace() / confusion_mat.sum()
    if best_prec < prec:
      best_prec = prec
      best_b = b

# Dual with kernel trick

In [ ]:
def kernel_mat(xi, xj):
  return (jnp.dot(xi, xj) + np.ones((N, N)))**3

def kernel_vec(xi, xj):
  return (np.dot(xi, xj) + 1)**3

def lag_kernel(c):
  partial_1 = -jnp.sum(c)

  c_outer = jnp.outer(c, c)
  c_outer = jnp.triu(c_outer)

  y_outer = jnp.outer(in_labels, in_labels)
  y_outer = jnp.triu(y_outer)

  K= kernel_mat(in_data, in_data.T)

  partial_2= 0.5 * jnp.sum(c_outer * y_outer * K)

  return partial_1 + partial_2

In [ ]:
lag_k_jit=jax.jit(lag_kernel)

linear_constraint = opt.LinearConstraint(in_labels, 0, 0, keep_feasible=True)
res = opt.minimize(lag_k_jit, a, method='trust-constr', jac='2-point', hess=hessian, constraints=[linear_constraint], options={'verbose': 1, 'maxiter': 1000}, bounds=opt.Bounds(np.zeros(200), np.ones(200)*np.inf))

In [ ]:
c = np.array(res.x)
w_phi = np.zeros(N)

index_non_zero = -1
for j in range(N):
  #NB: valid e training
  w_phi[j] = np.sum(np.array([c[i]*in_labels[i]*kernel_vec(valid[j, :], training[i, :]) for i in range(N)]))
  if c[j]>0 and index_non_zero<0:
    index_non_zero = j

b = - in_labels[index_non_zero] + w_phi[index_non_zero]